# import package

In [1]:
import math
import h5py
import scipy
import random
import scipy.io as scio
from PIL import Image
from scipy import ndimage
from tensorflow.python.framework import ops

import time
import tensorflow as tf 
import numpy as np
import scipy.misc
import pydot
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import SVG

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import layer_utils
from keras.utils import plot_model
from keras.utils.data_utils import get_file
from keras.utils.vis_utils import model_to_dot

import keras.backend as K

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

%matplotlib inline
np.random.seed(1)



Using TensorFlow backend.


# 辅助函数

In [2]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def max_min_normalization(data_array):
    rows = data_array.shape[0]
    cols = data_array.shape[1]
    
    temp_array = np.zeros((rows,cols))
    col_min = data_array.min(axis=0)
    col_max = data_array.max(axis=0)

    for i in range(0,rows,1):
        for j in range(0,cols,1):
            temp_array[i][j] = (data_array[i][j]-col_min[j])/(col_max[j]-col_min[j])
    return temp_array


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches



# load data

In [3]:
"下载数据和标签"
d = scio.loadmat('data1.mat')
data  = d['data']
label = d['label']
print('data shape = ',data.shape)
print('label shape = ',label.shape)

"随机打乱数据和标签"
N = data.shape[0]
index = np.random.permutation(N)
data  = data[index,:,:]
label = label[index,:]

"对数据data升维度,并且标签 one-hot"
data = np.expand_dims(data, axis=3)
label=label-1
label = convert_to_one_hot(label,52).T
print(data.shape, label.shape)

"选取训练样本、测试样本"
N = data.shape[0]
num_train = round(N*0.9)
num_test  = N-num_train

X_train = data[0:num_train,:,:,:]
Y_train = label[0:num_train,:]
X_test  = data[num_train:N,:,:,:]
Y_test  = label[num_train:N,:]

print(" ")
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))



data shape =  (11322, 16, 10)
label shape =  (11322, 1)
(11322, 16, 10, 1) (11322, 52)
 
number of training examples = 10190
number of test examples = 1132
X_train shape: (10190, 16, 10, 1)
Y_train shape: (10190, 52)
X_test shape: (1132, 16, 10, 1)
Y_test shape: (1132, 52)


# 残差块：标准块、卷积块

In [4]:
# 1、identity block
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base   = 'bn'  + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    # save the input value 
    X_shortcut = X

    # first component of main path
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid',
               name=conv_name_base+'2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)

    # second component of main path
    X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same',
               name=conv_name_base+'2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid',
               name=conv_name_base+'2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2c')(X)

    # Final step
    # Add shortcut value to main path, and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

# 2、convolutional block
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the identity block 

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, 主路径中间的那个CONV的窗口形状
    filters -- python整数列表, 定义主路径每个CONV层中的滤波器的数量
    stage --整数，用于命名层，取决于他们在网络中的位置          阶段
    block --字符串/字符，用于命名层，取决于他们在网络中的位置   块
    s -- 整数，指定滑动的大小

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base   = 'bn'  + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    # save the input value 
    X_shortcut = X

    # first component of main path
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), padding='valid',
               name=conv_name_base+'2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)

    # second component of main path
    X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same',
               name=conv_name_base+'2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid',
               name=conv_name_base+'2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2c')(X)

    # shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid',
            name=conv_name_base+'1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)

    # Final step
    # Add shortcut value to main path, and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


# create model

In [5]:
def ResNet50_semg(input_shape=(16,10,1), classes=52):
    
    X_input = Input(input_shape)
        
    # stage 1
    X = Conv2D(filters=4, kernel_size=(3,3), strides=(1,1), padding='valid',
               name='conv1', kernel_initializer=glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2), strides=(2,2))(X)

    # stage 2
    X = convolutional_block(X, f=3, filters=[4,4,16], stage=2, block='a', s=1)
    X = identity_block(X, f=3, filters=[4,4,16], stage=2, block='b')
    X = identity_block(X, f=3, filters=[4,4,16], stage=2, block='c')

    # stage 3
    X = convolutional_block(X, f=3, filters=[8,8,32], stage=3, block='a', s=1)
    X = identity_block(X, f=3, filters=[8,8,32], stage=3, block='b')
    X = identity_block(X, f=3, filters=[8,8,32], stage=3, block='c')
    X = identity_block(X, f=3, filters=[8,8,32], stage=3, block='d')

    # stage 4
    X = convolutional_block(X, f=3, filters=[16,16,64], stage=4, block='a', s=1)
    X = identity_block(X, f=3, filters=[16,16,64], stage=4, block='b')
    X = identity_block(X, f=3, filters=[16,16,64], stage=4, block='c')
    X = identity_block(X, f=3, filters=[16,16,64], stage=4, block='d')
    X = identity_block(X, f=3, filters=[16,16,64], stage=4, block='e')
    X = identity_block(X, f=3, filters=[16,16,64], stage=4, block='f')

    # stage 5
    X = convolutional_block(X, f=3, filters=[16,16,64], stage=5, block='a', s=1)
    X = identity_block(X, f=3, filters=[16,16,64], stage=5, block='b')
    X = identity_block(X, f=3, filters=[16,16,64], stage=5, block='c')

    # fully connected output layer
    X = Flatten(name='flatten')(X)
    X = Dense(256,    activation='relu',    name='fc1')(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50_semg')
    
    return model
    
    

# train & evaluate

In [6]:
model = ResNet50_semg(input_shape = (16, 10, 1), classes = 52)  
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=1, batch_size=64)


preds_train = model.evaluate(X_train, Y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, Y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))



Epoch 1/1
10190/10190 [==============================] - 18s 2ms/step
Train Loss = 3.96643827614
Train Accuracy = 0.0909715407262
1132/1132 [==============================] - 2s 2ms/step
Test Loss = 3.98231555379
Test Accuracy = 0.0962897526765


# plot model

In [7]:
"打印模型图层细节"
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16, 10, 1)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 14, 8, 4)     40          input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 14, 8, 4)     16          conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 14, 8, 4)     0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

__________________________________________________________________________________________________
res4c_branch2c (Conv2D)         (None, 7, 4, 64)     1088        activation_30[0][0]              
__________________________________________________________________________________________________
bn4c_branch2c (BatchNormalizati (None, 7, 4, 64)     256         res4c_branch2c[0][0]             
__________________________________________________________________________________________________
add_10 (Add)                    (None, 7, 4, 64)     0           bn4c_branch2c[0][0]              
                                                                 activation_28[0][0]              
__________________________________________________________________________________________________
activation_31 (Activation)      (None, 7, 4, 64)     0           add_10[0][0]                     
__________________________________________________________________________________________________
res4d_bran